In [2]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
import lightgbm as lgb
from xgboost  import XGBClassifier
import xgboost as xgb
from sklearn.ensemble import RandomForestRegressor
import catboost as cb
  

In [3]:
train= pd.read_csv("DATA/train.csv")
test= pd.read_csv("data/test.csv")


In [5]:
train.drop(['ID','판매구역'],axis=1,inplace=True)
test.drop(['ID','판매구역'],axis=1,inplace=True)

In [6]:
train.head()

,생산년도,모델출시년도,브랜드,차량모델명,판매도시,주행거리,배기량,압축천연가스(CNG),경유,가솔린,하이브리드,액화석유가스(LPG),가격
0,2018,2014,skoda,fabia,KAT,85231,999,0,0,1,0,0,51.74
1,2010,2006,toyota,auris,RKO,135000,1598,0,0,1,0,0,41.47
2,2002,2002,mercedes-benz,clk-klasa,GNI,255223,1796,0,0,1,0,0,17.81
3,2006,2001,nissan,x-trail,EHX,238000,2184,0,1,0,0,0,18.20
4,2007,2007,fiat,bravo,OSW,251000,1910,0,1,0,0,0,17.55


In [7]:
cat_col1 =list(train.select_dtypes(include="object").columns)
cat_col1

['브랜드', '차량모델명', '판매도시']

In [8]:
def cat_dtyes (df):
    cat_col = df.select_dtypes(include="object").columns
    df[cat_col] = df[cat_col].astype("category")

    return df

In [9]:

train = cat_dtyes(train)
test = cat_dtyes(test)

In [10]:
y_train = train['가격']
X_train = train.drop(columns=['가격'])

In [11]:
X_train, X_val, y_train, y_val = train_test_split(X_train,y_train,
                                                    test_size=0.2, random_state=42)

In [12]:
print(X_train.shape, y_train.shape)
print(X_val.shape, y_val.shape)

(46336, 12) (46336,)
(11584, 12) (11584,)


In [13]:
model_cat2=cb.CatBoostRegressor(loss_function='MAE', verbose=0,cat_features=cat_col1)

In [16]:
distributions2 = {
    'depth': [10,20,30,50],
    'learning_rate':[0.01, 0.05, 0.1],
    'n_estimators':[2000, 2500],    
    'random_seed':[42, 43, 44, 45, 100],
    'l2_leaf_reg':[0.1, 0.3, 0.4, 0.5, 1]
    
}

In [17]:
randomized_search_result2 = model_cat2.randomized_search(
    distributions2,
    X = X_train, y = y_train, 
    cv=5, n_iter=20,
    plot=True
)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))


bestTest = 5.988234502
bestIteration = 1998

0:	loss: 5.9882345	best: 5.9882345 (0)	total: 36.6s	remaining: 11m 35s

bestTest = 5.907382064
bestIteration = 1108

1:	loss: 5.9073821	best: 5.9073821 (1)	total: 1m 23s	remaining: 12m 28s

bestTest = 5.914539041
bestIteration = 1999

2:	loss: 5.9145390	best: 5.9073821 (1)	total: 1m 58s	remaining: 11m 13s

bestTest = 5.954809589
bestIteration = 1961

3:	loss: 5.9548096	best: 5.9073821 (1)	total: 2m 34s	remaining: 10m 18s

bestTest = 5.960688109
bestIteration = 2496

4:	loss: 5.9606881	best: 5.9073821 (1)	total: 3m 19s	remaining: 9m 57s

bestTest = 5.952957952
bestIteration = 2498

5:	loss: 5.9529580	best: 5.9073821 (1)	total: 4m 5s	remaining: 9m 32s

bestTest = 5.957922226
bestIteration = 1038

6:	loss: 5.9579222	best: 5.9073821 (1)	total: 4m 50s	remaining: 8m 59s
Estimating final quality...
Training on fold [0/5]

bestTest = 5.942990412
bestIteration = 618

Training on fold [1/5]

bestTest = 5.988887373
bestIteration = 1035

Training on fo

In [18]:
randomized_search_result2

{'params': {'depth': 10,
  'random_seed': 43,
  'iterations': 2500,
  'learning_rate': 0.1,
  'l2_leaf_reg': 0.4},
 'cv_results': defaultdict(list,
             {'iterations': [0,
               1,
               2,
               3,
               4,
               5,
               6,
               7,
               8,
               9,
               10,
               11,
               12,
               13,
               14,
               15,
               16,
               17,
               18,
               19,
               20,
               21,
               22,
               23,
               24,
               25,
               26,
               27,
               28,
               29,
               30,
               31,
               32,
               33,
               34,
               35,
               36,
               37,
               38,
               39,
               40,
               41,
               42,
               43,
            

In [27]:
model_cat=cb.CatBoostRegressor(
    loss_function='MAE', verbose=0,cat_features=cat_col1, depth =10,
    random_seed = 44,
    l2_leaf_reg= 1,
    iterations =2000,
    learning_rate =0.05)



In [28]:
model_cat.fit(X_train,y_train)

In [29]:
pred_cat2= model_cat.predict(X_val)

In [30]:
mean_absolute_error(y_val, pred_cat2)

5.857053122537162

In [31]:
pred = model_cat.predict(test)

In [32]:
sample_submission = pd.read_csv('data/sample_submission.csv')
sample_submission['가격'] = pred
sample_submission.to_csv("submit_cat_h2.csv", index = False)